# Unit Converter - Linear Array Approach

This is a unit converter that takes three inputs, a value, the values unit, and the desired output unit. It fits under the linear array aproach for unit converters as it calculates how to convert between units by hunting for the original and desired units in an array. Once found, provided the units are compatible, it calculates the product of all the conversion rates between the two units, and finally multiplies the original value by this product.

In [1]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import math
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logging.getLogger().setLevel(logging.INFO) 

#### Defining reusable functions that I have made previously

In the following cell are functions that I have previously created for reocurring problems. These are: 
1. sigfigs_str which takes a string input (string) assuming it contains a number, and calculates the significant figures to which the number is given.
2. round_sf which takes a float and an integer input (x,sf) and rounds the float to the number of significant figures specified by the integer.
3. floatconv takes a string input (instr) and converts it to a float if the string is not problematic (ie has max. 1 decimal and no other non-digit characters). If the string is problematic, an error is raised depending on whether its a decimal error or a non-digit character error.
4. find which takes a string input (target) and an array name (array) and finds the target string in the array even if there are nested arrays. The find function will raise an error if the target string is not found in the array.

In [2]:
#defining a function that will calculate the significant figures of a number given as a string
def sigfigs_str(string):
    if "." in string:
        digits = string.replace(".", "").lstrip("0")
    else:
        digits = string.lstrip("0")
    logging.debug(f"sigfigs_str called with string = '{string}' digits = '{digits}'")
    if not digits:
        return 0
    return len(digits)

#defining a function that will round based on significant figures rather than decimals
def round_sf(x, sf):
    
    if x == 0:
        return 0
    else:
        rounded = round(x, sf - int(math.floor(math.log10(abs(x)))+1))
        logging.debug(f"round_sf called with x = '{x}' sf = '{sf}' giving rounded = '{rounded}'")
        return rounded

#defining a float conversion function to take string input and return a float or an error
def floatconv(instr):
    decimal_seen = False
    out = 0.0
    for i, char in enumerate(instr):
        if char.isdigit():
            continue
        elif char == '.':
            if decimal_seen:
                raise ValueError(f"Error: too many decimals in input '{instr}'")
            decimal_seen = True
        else:
            raise ValueError(f"Error: non-numerical character '{char}' in input '{instr}'")
    logging.debug(f"floatconv called with instr = '{instr}' giving out = '{float(instr)}'")
    return float(instr)

#defining a find function to find a target in an array that may have arrays in it
def find(array, target, path=None, top=True):
    if path is None:
        path = []
    for i, item in enumerate(array):
        if isinstance(item, list):
            result = find(item, target, path + [i], top=False)
            if result is not None:
                return result
        elif item == target:
            path += [i]
            logging.debug(f"find called with target = '{target}' giving path = '{path}'")
            return path
    if top:
        raise IndexError(f"Error: input unit '{target}' not found")
    return None

#### Defining the convert function

The following cell contains the convert function that is specific to unit conversion. Within the convert function lies a cal (calculate) function that does the actual maths of converting units. The cal function is only called if units are compatible (i.e a weight to a weight) and if the original unit and the desired unit arent the same. If the units arent compatible an error is raised to testify to this. All the functions in the previous cell are called up for formatting inputs and outputs to the correct type and to suitable length (ie rounding). As you can see, the available units and conversions between units are hard codded into the convert function at the start. They could be extracted from a csv file and it would be more suitable to format them as one nested dictionary rather than two nested arrays (Units and Conversions), but I havent as this is basic file handling and formating and that was not what I was practicing by making this function. You can also see that each nested array in Conversions is one element shorter than it corresponding nested array in Units. This is as each value in Conversions, is the multiplication factor you would need to go from the corresponding position in Units, to the one on its immediate right (confusing I know). For example in Weights, you would have to multiply a value measured in st (4th element in Weights) by 14.0 (4th element in Weightsconv) to get the equivalent weight in lb (5th element in Weights).

In [3]:
#defining a convert function to convert the input value from the input units to the output units
def convert(invalstr,inunit,outunit):
    
    #defining units and their conversion rates
    Weights = ["tonne","kg","g","st","lb","oz"]
    Distances = ["km","m","cm","mm","mi","yd","ft","in"]
    Volumes = ["m3","l","cc","ml","yd3","ft3","in3","gal","qt","pt","c","floz","tbsp","tsp"]
    Units = [Weights,Distances,Volumes]
    Weightsconv = [1000.0,1000.0,0.0001574730901,14.0,16.0]
    Distancesconv = [1000.0,100.0,10.0,0.0000006213711922,1760.0,3.0,12.0]
    Volumesconv = [1000.0,1000.0,1.0,0.000001307950619,27.0,1728.0,0.004329004329,4.0,2.0,2.0,8.0,2.0,3.0]
    Conversions = [Weightsconv,Distancesconv,Volumesconv]

    #defining the internal calculation function
    def cal(inval,inunitpos,outunitpos):
        fac = 1.0
        x = 0
        while x < abs(outunitpos[1]-inunitpos[1]):
            array = Conversions[inunitpos[0]]
            if inunitpos[1]<outunitpos[1]:
                fac = fac*array[inunitpos[1]+x]
            elif inunitpos[1]>outunitpos[1]:
                fac = fac/array[outunitpos[1]+x]
            x = x + 1
        outval = inval * fac
        logging.debug(f"cal called with inval = '{inval}' inunitpos = '{inunitpos}' outunitpos = '{outunitpos}' giving outval = '{outval}'")
        return outval
    
    #processing inputs
    inval = floatconv(invalstr)
    sigfig = sigfigs_str(invalstr)
    #allowing minimum sigfig of 3 even if fewer were given
    if sigfig<3:
        sigfig = 3
    #finding units
    inunitpos = find(Units,inunit)
    outunitpos = find(Units,outunit)

    #comparing units for compatibility
    if inunitpos[0] != outunitpos[0]:
        raise KeyError(f"Error: incompatible units '{inunit}' and '{outunit}'")
    #avoiding processing if input unit and output unit are the same
    elif inunitpos == outunitpos:
        print("You have entered the same units.")
        outval = inval
    #processing everything else
    else:
        outval = round_sf(cal(inval,inunitpos,outunitpos),sigfig)

    #giving the final output
    print(inval,inunit," is equal to ",outval,outunit," to ",sigfig,"significant figures.")

#### Calling the overall  convert function

Here the function can be called. There are a few examples of how the function handles potentially problematic inputs, and the final cell can be run to given inputs in the notebook. 

In [4]:
#Example: this example is just simple and as the function is intended to be used
convert("5","lb","kg")

5.0 lb  is equal to  2.27 kg  to  3 significant figures.


In [5]:
#calling convert
val = input("Please input the value you want to convert:")
inunit = input("Please input the original unit:")
outunit = input("Please input the desired unit:")
logging.getLogger().setLevel(logging.INFO)
#logging.getLogger().setLevel(logging.DEBUG) """un-comment to run debug"""
convert(val,inunit,outunit)

Please input the value you want to convert: 127
Please input the original unit: in
Please input the desired unit: m


127.0 in  is equal to  3.23 m  to  3 significant figures.
